In [1]:
import pandas as pd
import numpy as np

In [2]:
pheno = pd.read_csv('../16p12.2_rnaseq_analysis/data/pheno_final.tsv', sep='\t')

samples = list(pheno.subject.unique())

In [3]:
with open('survivor/all_codes.list', 'r') as f:
    allsamples = f.readlines()
allsamples = [s.strip() for s in allsamples]

# Gene Annotation


In [4]:
gdf = pd.read_csv('refGene_hg19.txt', sep='\t', header=None)

gdf['chrom'] = gdf[2]
gdf['start'] = gdf[4]
gdf['end']   = gdf[5]
gdf['gene']  = gdf[12]
gdf = gdf[['chrom', 'start', 'end', 'gene']]

In [5]:
for subject in allsamples:
#     print(subject)
    df = pd.read_csv('output/merged/{}.merged.tsv'.format(subject), sep='\t')
    df = df[df.length < 50e3]
    df['genes'] = ''

    for chrom in df.chrom.unique():
    #     print(chrom)
        dfc = df[df.chrom == chrom]
        gdfc = gdf[gdf.chrom == chrom]
        for i in dfc.index[:]:
            start = dfc.at[i, 'start']
            end = dfc.at[i, 'end']
            min_end   = gdfc.end.apply(lambda x: min(x, end))
            max_start = gdfc.start.apply(lambda x: max(x, start))
            odf = gdfc[(min_end - max_start) > 0]
            if odf.shape[0] > 0:
                df.at[i, 'genes'] = ';'.join(odf.gene.tolist())

    df.to_csv('output/merged/{}.merged.genes.tsv'.format(subject), sep='\t', index=False)

# inter cohort frequency



In [6]:
cdf = pd.read_csv('gnomad_v2.1_controls_dup_del.bed', sep='\t', header=None)
cdf['chrom'] = cdf[0].apply(lambda s: 'chr{}'.format(s))
cdf['start'] = cdf[1]
cdf['end'] = cdf[2]
cdf['svtype'] = cdf[4]
cdf['freq'] = cdf[5] 
cdf = cdf[['chrom', 'start', 'end', 'svtype', 'freq']]
cdf['length'] = cdf.end - cdf.start

In [7]:
for subject in samples:

#     print(subject)

    df = pd.read_csv('output/merged/{}.merged.genes.tsv'.format(subject), sep='\t')

    df['gnomad_freq'] = 0.

    for chrom in df.chrom.unique()[:]:
        for svtype in ['DUP', 'DEL']:
            dfc = df[(df.chrom == chrom) & (df.svtype == svtype)]
            cdfc = cdf[(cdf.chrom == chrom) & (cdf.svtype == svtype)]

            for i in dfc.index[:]:
                start = dfc.at[i, 'start']
                end = dfc.at[i, 'end']
                length = dfc.at[i, 'length']
                min_end   = cdfc.end.apply(lambda x: min(x, end))
                max_start = cdfc.start.apply(lambda x: max(x, start))
                max_length = cdfc.length.apply(lambda x: max(x, length))
            #     print(i, start, end, length)
                odf = cdfc[(min_end - max_start) > .5 * max_length]
                if odf.shape[0] > 0:
                    freq = odf.freq.mean()
                    df.at[i, 'gnomad_freq'] = freq

    df.to_csv('output/merged/{}.merged.inter_cohort.tsv'.format(subject), sep='\t', index=False)

# intra cohort frequency

In [8]:
subject = allsamples[0]

df = pd.read_csv('output/merged/{}.merged.tsv'.format(subject), sep='\t')
dfall = pd.DataFrame(columns = df.columns)

for subject in allsamples:
#     print(subject)
    dfa = pd.read_csv('output/merged/{}.merged.tsv'.format(subject), sep='\t')
    dfa['subject'] = subject
    dfall = dfall.append(dfa)

In [9]:
dfall = dfall.reset_index(drop=True)
# dfall

In [10]:
for subject in allsamples[:]:

    print(subject)
    df = pd.read_csv('output/merged/{}.merged.genes.tsv'.format(subject), sep='\t')
    rdf = dfall[dfall.subject != subject]
    rdf = rdf.sort_values(['chrom', 'start'])


    df['intra_cohort_count'] = 0

    for chrom in df.chrom.unique()[:]:
        for svtype in ['DUP', 'DEL']:
            dfc = df[(df.chrom == chrom) & (df.svtype == svtype)]
            rdfc = rdf[(rdf.chrom == chrom) & (rdf.svtype == svtype)]

            rstarts = rdfc.start.to_numpy()
            rends = rdfc.end.to_numpy()
            rlengths = rdfc.length.to_numpy()
            
            for i in dfc.index[:]:
                start = dfc.at[i, 'start']
                end = dfc.at[i, 'end']
                length = dfc.at[i, 'length']
                min_end    = np.minimum(rends, end)
                max_start  = np.maximum(rstarts, start)
                max_length = np.maximum(rlengths, length)
                overlap = (min_end - max_start) > .5 * max_length
                if overlap.sum() > 0:
                    odf = rdfc[overlap]
                    count = len(odf.subject.unique())
                    df.at[i, 'intra_cohort_count'] = count

    df['intra_cohort_freq'] = df['intra_cohort_count']/ 345.
    df.to_csv('output/merged/{}.annotated.tsv'.format(subject), sep='\t', index=False)

# inheritence patterns

In [11]:
rel = pd.read_csv('allfamilies.ped', sep='\t')
rel = rel.set_index('Sample_ID', drop=False)
rel = rel[rel.Sample_ID.isin(samples)]
# rel.head()

In [12]:
for subject in rel.Sample_ID:


    father = rel.at[subject, 'Father']
    mother = rel.at[subject, 'Mother']
    if father == '0' or mother == '0' or subject == 'SG011':
        print(subject, '0')
        df = pd.read_csv('output/merged/{}.annotated.tsv'.format(subject), sep='\t')
        df['inh_from_father'] = ''
        df['inh_from_mother'] = ''
        df.to_csv('output/merged/{}.annotated.tsv'.format(subject), sep='\t', index=False)
        continue
#     print(subject, father, mother)

    df = pd.read_csv('output/merged/{}.annotated.tsv'.format(subject), sep='\t')
    fdf = pd.read_csv('output/merged/{}.annotated.tsv'.format(father), sep='\t')
    mdf = pd.read_csv('output/merged/{}.annotated.tsv'.format(mother), sep='\t')

    df['inh_from_father'] = ''
    df['inh_from_mother'] = ''

    for chrom in df.chrom.unique()[:]:
        for svtype in ['DUP', 'DEL']:
            dfc = df[(df.chrom == chrom) & (df.svtype == svtype)]
            fdfc = fdf[(fdf.chrom == chrom) & (fdf.svtype == svtype)]
            mdfc = mdf[(mdf.chrom == chrom) & (mdf.svtype == svtype)]

            for i in dfc.index[:]:
                start = dfc.at[i, 'start']
                end = dfc.at[i, 'end']
                length = dfc.at[i, 'length']
                min_end   = fdfc.end.apply(lambda x: min(x, end))
                max_start = fdfc.start.apply(lambda x: max(x, start))
                max_length = fdfc.length.apply(lambda x: max(x, length))
                overlap = (min_end - max_start) > .5 * max_length
                if overlap.sum() > 0:
                    df.at[i, 'inh_from_father'] = 'X'

                min_end   = mdfc.end.apply(lambda x: min(x, end))
                max_start = mdfc.start.apply(lambda x: max(x, start))
                max_length = mdfc.length.apply(lambda x: max(x, length))
                overlap = (min_end - max_start) > .5 * max_length
                if overlap.sum() > 0:
                    df.at[i, 'inh_from_mother'] = 'X'

    df.to_csv('output/merged/{}.annotated.tsv'.format(subject), sep='\t', index=False)

# combine into one file

In [13]:

df = pd.read_csv('output/merged/{}.annotated.tsv'.format(subject), sep='\t')
dfall = pd.DataFrame(columns = df.columns)

for subject in allsamples:
#     print(subject)
    dfa = pd.read_csv('output/merged/{}.annotated.tsv'.format(subject), sep='\t')
    dfa['subject'] = subject
    dfall = dfall.append(dfa)

In [14]:
dfall.to_csv('output/merged/allsamples.annotated.tsv', sep='\t', index=False)

# filter < 10 counts

In [15]:
cnvs = pd.read_csv('output/merged/allsamples.annotated.tsv', sep='\t')
cnvs = cnvs[cnvs.intra_cohort_count < 10]

In [16]:
cnvs.to_csv('output/merged/allsamples.annotated.filtered.tsv', sep='\t', index=False)

# annotate ensembl

In [4]:
cnvs = pd.read_csv('output/merged/rna_samples.annotated.filtered.tsv', sep='\t')

rel = pd.read_csv('../16p12.2_rnaseq_analysis/data/family_summaries.tsv', sep='\t')
rel = rel[rel.subject != 'SG011']
rel = rel.set_index('subject', drop=False)

keep = pd.read_csv('../16p12.2_rnaseq_analysis/outlier_expression_analysis/keep.tsv', sep='\t')

mapp = pd.read_csv('../16p12.2_rnaseq_analysis/data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl')
mapp = mapp.loc[keep.ensembl]
mapp['chrom'] = mapp.chromosome.apply(lambda s: 'chr{}'.format(s) if s != 'MT' else 'chrM')
mapp.start = mapp.start.astype(int)
mapp.end = mapp.end.astype(int)

mapp['ensembl'] = mapp.Name.apply(lambda s: s.split('.')[0])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
cnvs['ensembl'] = ''
for i in cnvs.index[:]:
    chrom = cnvs.at[i, 'chrom']
    start = cnvs.at[i, 'start']
    end = cnvs.at[i, 'end']

    mappc = mapp[mapp.chrom == chrom]

    min_end = mappc.end.apply(lambda x: min(x, end))
    max_start = mappc.start.apply(lambda x: max(x, start))

    overlap = (min_end - max_start) > 0

    if overlap.sum() > 0:
        mappo = mappc[overlap]  
#         print(i, chrom, start, end, mappo.shape)

        for j in mappo.index:
            if cnvs.at[i, 'ensembl'] == '':
                cnvs.at[i, 'ensembl'] =  mappo.at[j, 'ensembl'] 
            else:
                cnvs.at[i, 'ensembl'] = cnvs.at[i, 'ensembl'] + ';'+ mappo.at[j, 'ensembl'] 

In [32]:
cnvs.to_csv('output/merged/rna_samples.annotated.filtered.tsv', sep='\t', index=False)

# group svs

In [36]:
cnvs = pd.read_csv('output/merged/rna_samples.annotated.filtered.tsv', sep='\t')

keep = pd.read_csv('../16p12.2_rnaseq_analysis/outlier_expression_analysis/keep.tsv', sep='\t')

mapp = pd.read_csv('../16p12.2_rnaseq_analysis/data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl')
mapp = mapp.loc[keep.ensembl]
mapp['chrom'] = mapp.chromosome.apply(lambda s: 'chr{}'.format(s) if s != 'MT' else 'chrM')
mapp.start = mapp.start.astype(int)
mapp.end = mapp.end.astype(int)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
cnvs = cnvs[~(cnvs['ensembl'].isna())]

In [53]:
by_gene = []

for i, row in cnvs.iterrows():
    if ';' not in row.ensembl:
        by_gene.append(row.to_list())
    else:
        for gene in row.ensembl.split(';'):
            to_append = row.to_list()
            to_append[-1] = gene
            by_gene.append(to_append)
            
by_gene = pd.DataFrame(by_gene, columns=cnvs.columns)


In [68]:
by_gene = by_gene.set_index('ensembl', drop=False)

In [70]:
by_gene['gene_start'] = mapp.loc[by_gene.ensembl]['start']
by_gene['gene_end'] = mapp.loc[by_gene.ensembl]['end']
by_gene['gene_strand'] = mapp.loc[by_gene.ensembl]['strand']

In [72]:
def group_svs(sv_start, sv_end, gene_start, gene_end, gene_strand):
    if sv_start < gene_start and gene_end < sv_end:
        return 'encapsulated'
    if gene_start < sv_start and sv_end < gene_end:
        return 'interstitial'
    if gene_strand == '+':
        if sv_start < gene_start and sv_end < gene_end:
            return '5prime'
        if sv_start > gene_start and sv_end > gene_end:
            return '3prime'
    if gene_strand == '-':
        if sv_start < gene_start and sv_end < gene_end:
            return '3prime'
        if sv_start > gene_start and sv_end > gene_end:
            return '5prime'
        
    return 'todo'

by_gene['group'] = [group_svs(s[0], s[1], s[2], s[3], s[4]) for s in zip(by_gene['start'], by_gene['end'], by_gene['gene_start'], by_gene['gene_end'], by_gene['gene_strand'])]

In [74]:
by_gene.to_csv('output/merged/rna_samples.annotated.filtered.by_gene.tsv', sep='\t', index=False)